# 常用find查询操作

### 查询指定字段最新一条记录

res = movie.find({}).sort('_id', -1).limit(1)

**注意: ** limit返回的是一个迭代器,并不是数据结果

###  嵌套查询

In [ ]:
# 查询doc中key为response且response中key为status_code=302 记录的数目
db.getCollection('urls').find({'response.status_code': 302}).count()

### 查询返回部分字段,不返回整个文档

In [ ]:
查询返回部分字段,不返回整个文档可以节省流量,降低带宽

* 第一个参数为查询条件，空代表查询所有文档
* 第二个参数中 1 代表选取该字段的值, 0代表过滤该字段的值

如果需要输出的字段比较多，不想要某个字段，可以用排除字段的方法 

# 返回所有含有title和type字段的值,并不是包含这些字段的所有文档
db.news.find( {}, {'title': 1,  'type': 1} )

# 查询返回
db.news.find( {}, {'title': 1,  'type': 0} )

db.inventory.find( { type: 'food' }, { type:0 })


# 不输出内容字段，其它字段都输出
db.news.find( {}, {content: 0 } )

### 分页查询

In [ ]:
db.collection.find({}).slip(index).limit(num)

### 范围查询或值大小比较过滤

范围查询用到: `=, !=, >, >=, <, <=`这6种
```
cursor = db['sys'].find({'token': token, 'time': {'$gt': start}}, {'_id': 0, 'average': 1, 'time': 1})
cursor = db['sys'].find({'token': token, 'time': {'$lt': end}}, {'_id': 0, 'average': 1, 'time': 1})
cursor = db['sys'].find({'token': token, 'time': {'$gte': start}}, {'_id': 0, 'average': 1, 'time': 1})
cursor = db['sys'].find({'token': token, 'time': {'$lte': end}}, {'_id': 0, 'average': 1, 'time': 1})
cursor = db['sys'].find({'token': token, 'time': {'$gt': start, '$lt': end}}, {'_id': 0, 'average': 1, 'time': 1})
cursor = db['sys'].find({'token': token, 'time': {'$ne': start}}, {'_id': 0, 'average': 1, 'time': 1})
```

### 去重查询

In [ ]:
# 简单去重方法,适合数据量不大的情况下, 返回去重后的列表
db.collection.distinct("page")

### 统计不含有某一字段的记录数

In [ ]:
dbName.collectionName.find({fieldName:null}).count()

### 查询给定字段有重复数据的记录

In [ ]:
db.collection.aggregate([
  { $group: { 
    _id: { page: "$page", secondField: "$secondField" }, 
    uniqueIds: { $addToSet: "$_id" },
    count: { $sum: 1 } 
  }}, 
  { $match: { 
    count: { $gt: 1 } 
  }}
])

### 查询两个字段值相同的记录

In [ ]:
db.foo.find({"$where":function(){
 for(var current in this){
   for(var other in this){
     if(current != other && this[current] == this[other]){
       return true;
     }
   }
 }
 return false;
}})

### 正则匹配查询或模糊匹配

MongoDB中对子串的搜索只有正则表达式一种方法 并没有SQL中的LIKE语句

PyMongo的正则使用方法与shell中的不大一致 方法如下所示 $options是正则的选项 i就是不区分大小写

find({"name":{"$regex":regex, "$options":"i"}})


1. import re
{'xxx':re.compile('xxx')}

2. {'xxx':{'$regex':'xxx'}}


### 查询某一字段不是数字的记录
db.getCollection('phone').find({"tel": {"$regex": '^[^0-9]+$'}})

### 查询某一字段不是数字的记录的数目
db.getCollection('phone').find({"tel": {"$regex": '^[^0-9]+$'}}).count()

### 删除某一字段不是数字的记录
db.getCollection('phone').remove({"tel": {"$regex": '^[^0-9]+$'}})

### 根据字段排序

pymongo中排序(默认为升序):
* 升序: pymongo.ASCENDING = 1
* 降序: pymongo.DESCENDING = -1

单个字段排序:

```
db.Account.find().sort("UserName")  --默认为升序
db.Account.find().sort("UserName",pymongo.ASCENDING)   --升序
db.Account.find().sort("UserName",pymongo.DESCENDING)  --降序
```

多字段排序:
```
db.Account.find().sort([("UserName", pymongo.ASCENDING),("Email",pymongo.DESCENDING)])
```

### 使用多个逻辑或(or)

多个字段匹配, 其中:

* token在有限集合中匹配, score是范围匹配
* account, CLI, PID这3个字段支持关键字正则匹配

In [ ]:
queries = []
for token, threshold in all_threshold.items():
    queries.append({'token': token, 'score': {'$lt': threshold}})

filters = {'$or': queries, '$and':[{'$or':[{"account": regex_kw}, {"CLI": regex_kw}, {"PID": regex_kw}]}]}

# db.things.find({$and: [{$or : [{'a':1},{'b':2}]},{$or : [{'a':2},{'b':3}]}] })